# NB from the scratch


In [2]:
import numpy as np


class GaussianNaiveBayes:

  def fit(self, X, y):
    n_samples, n_features = X.shape
    self._classes = np.unique(y)
    n_classes = len(self._classes)
    self._mean = np.zeros((n_classes, n_features), dtype=np.float64)
    self._var = np.zeros((n_classes, n_features), dtype=np.float64)
    self._priors = np.zeros(n_classes, dtype=np.float64)

    # calculating the mean, variance and prior P(H) for each class
    for i, c in enumerate(self._classes):
      X_for_class_c = X[y == c]
      self._mean[i, :] = X_for_class_c.mean(axis=0)
      self._var[i, :] = X_for_class_c.var(axis=0)
      self._priors[i] = X_for_class_c.shape[0] / float(n_samples)

  def _calculate_likelihood(self, class_idx, x):
    mean = self._mean[class_idx]
    var = self._var[class_idx]
    num = np.exp(-(x - mean)**2 / (2 * var))  # numerator
    denom = np.sqrt(2 * np.pi * var)  # denominator
    return num / denom

  def predict(self, X):
    y_pred = [self._classify_sample(x) for x in X]
    return np.array(y_pred)

  def _classify_sample(self, x):
    posteriors = []
    # calculating posterior probability for each class
    for i, c in enumerate(self._classes):
      prior = np.log(self._priors[i])
      posterior = np.sum(np.log(self._calculate_likelihood(i, x)))
      posterior = prior + posterior
      posteriors.append(posterior)
    # return the class with highest posterior probability
    return self._classes[np.argmax(posteriors)]

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import datasets
import time

X, y = datasets.make_classification(n_samples=1000,
                                    n_features=20,
                                    n_classes=2,
                                    random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.25,
                                                    random_state=42)

start = time.perf_counter()
nb = GaussianNaiveBayes()
nb.fit(X_train, y_train)
predictions = nb.predict(X_test)
end = time.perf_counter()
print(f"NumPy Naive Bayes accuracy: {accuracy_score(y_test, predictions)}")
print(f'Finished in {round(end-start, 3)} second(s)')

NumPy Naive Bayes accuracy: 0.796
Finished in 0.013 second(s)


In [4]:
from sklearn.naive_bayes import GaussianNB

start = time.perf_counter()
sk_nb = GaussianNB()
sk_nb.fit(X_train, y_train)
sk_predictions = sk_nb.predict(X_test)
end = time.perf_counter()
print(
    f"scikit-learn Naive Bayes accuracy: {accuracy_score(y_test, sk_predictions)}"
)
print(f'Finished in {round(end-start, 3)} second(s)')

scikit-learn Naive Bayes accuracy: 0.796
Finished in 0.002 second(s)
